<a href="https://colab.research.google.com/github/satishgaurav/Molecule-Classification/blob/main/Copy_of_chemprop_colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import userdata
# userdata.get('github')

In [ ]:
!git clone https://{userdata.get('github')}@github.com/satishgaurav/Molecule-Classification.git

Cloning into 'Molecule-Classification'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 39 (delta 15), reused 6 (delta 2), pack-reused 0
Receiving objects: 100% (39/39), 5.57 MiB | 5.87 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [ ]:
!cp ./Molecule-Classification/*.csv ./

In [ ]:
!ls -alh

total 9.8M
drwxr-xr-x 1 root root 4.0K Mar 11 10:18 .
drwxr-xr-x 1 root root 4.0K Mar 11 10:10 ..
-rw-r--r-- 1 root root 146K Mar 11 10:18 BBBP.csv
drwxr-xr-x 4 root root 4.0K Mar  7 14:31 .config
-rw-r--r-- 1 root root  61K Mar 11 10:18 fda_approved.csv
-rw-r--r-- 1 root root 2.1M Mar 11 10:18 HIV.csv
drwxr-xr-x 4 root root 4.0K Mar 11 10:15 Molecule-Classification
-rw-r--r-- 1 root root 4.5M Mar 11 10:18 musk.csv
-rw-r--r-- 1 root root 3.1M Mar 11 10:18 named.csv
drwxr-xr-x 1 root root 4.0K Mar  7 14:32 sample_data
-rw-r--r-- 1 root root 2.8K Mar 11 10:18 substances.csv


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Mar 11 10:19:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
!pip install chemprop
!pip install rdkit-pypi  # should be included in above after Chemprop v1.6 release

import chemprop
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 44.3 MB/s eta 0:00:00
  Created wheel for typed-argument-parser: filename=typed_argument_parser-1.9.0-py3-none-any.whl size=25615 sha256=43e41f37ab5618f8650931d9a6559216a8627d8456f264162fb4aae6770513d1
  Stored in directory: /root/.cache/pip/wheels/f0/94/0f/9539f578bed7e1bd423c702e403712f5ee8989f831a71db000
Successfully built typed-argument-parser
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 32.4 MB/s eta 0:00:00


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
hiv_df = pd.read_csv("HIV.csv")
hiv_df.head()

,smiles,activity,HIV_active
0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,CI,0
1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,CI,0
2,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,CI,0
3,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,CI,0
4,O=S(=O)(O)CCS(=O)(=O)O,CI,0


In [ ]:
hiv_df.describe()

,HIV_active
count,41127.000000
mean,0.035086
std,0.184001
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
unique_values = hiv_df['HIV_active'].unique()
print(f"Unique values in 'HIV_active': {unique_values}")

Unique values in 'HIV_active': [0 1]


In [ ]:
unique_values = hiv_df['activity'].unique()
print(f"Unique values in 'activity': {unique_values}")

Unique values in 'activity': ['CI' 'CM' 'CA']


In [ ]:
unique_values = hiv_df['smiles'].unique()
print(f"Unique values in 'smiles': {unique_values}")
print(f"length of uniqe value: {len(unique_values)}")

Unique values in 'smiles': ['CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2'
 'C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3)CC(c3ccccc3)=[O+]2)[O+]=C(c2ccccc2)C1'
 'CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21' ...
 'Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)cc1'
 'Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C)CC4C3C2=O)c1'
 'CCCCCC=C(c1cc(Cl)c(OC)c(-c2nc(C)no2)c1)c1cc(Cl)c(OC)c(-c2nc(C)no2)c1']
length of uniqe value: 41127


In [ ]:
# Filter rows where 'your_column' is not equal to 1 or 0
filtered_df = hiv_df[(hiv_df['HIV_active'] != 1) & (hiv_df['HIV_active'] != 0)]
filtered_df

,smiles,activity,HIV_active


In [ ]:
# Filter rows where 'target_column' is equal to 1h
hiv_df_filtered_active = hiv_df[hiv_df['HIV_active'] == 1]
hiv_df_filtered_active

,smiles,activity,HIV_active
11,O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1,CM,1
16,NNP(=S)(NN)c1ccccc1,CM,1
80,O=Nc1ccc(O)c(N=O)c1O,CM,1
203,Oc1ccc(Cl)cc1C(c1cc(Cl)ccc1O)C(Cl)(Cl)Cl,CM,1
234,NNC(=O)c1ccccc1SSc1ccccc1C(=O)NN,CM,1
...,...,...,...
41090,Cc1cn(COCCCOCC(=O)c2ccccc2)c(=O)[nH]c1=O,CM,1
41092,Cc1cn(C2CC3C(COC(CCC[Se]c4ccccc4)N3O)O2)c(=O)[...,CM,1
41093,Cc1cn(C2CC3C(COC(CCCC[Se]c4ccccc4)N3O)O2)c(=O)...,CM,1
41098,Cc1cn(C2CC3C(COC(CC[Se]C#N)N3O)O2)c(=O)[nH]c1=O,CM,1


In [ ]:
# Filter rows where 'target_column' is equal to 1h
hiv_df_filtered_inactive = hiv_df[hiv_df['HIV_active'] == 0]
hiv_df_filtered_inactive = hiv_df_filtered_inactive.sample(n=1500, axis=0, random_state=42)
hiv_df_filtered_inactive

,smiles,activity,HIV_active
2428,O=C1c2ccccc2-c2nc3ccccc3nc21,CI,0
6197,O=C(CSc1cc(-c2ccc(Cl)cc2)s[s+]1)c1ccccc1,CI,0
17138,O=C(C=Nc1ccccc1C(=O)O)c1ccco1,CI,0
12261,CCCCCCCCCCCCCCCCCC[N+](C)(C)Cc1ccc(C[N+](C)(C)...,CI,0
3588,N#CSC1CCCCCCC1SC#N,CI,0
...,...,...,...
18477,CC(=O)OC1(C#N)CC2OC1C1C2N1C(=O)OC(C)(C)C,CI,0
1189,CCOC(=O)C1Cc2cc(C)c(C)cc2N(C)C1=O,CI,0
36657,CCOC(=O)N1CCN(c2ccc3c(C)cc(C)nc3n2)CC1,CI,0
27919,CN(C)C=Nc1ccc2c3c(cccc13)-c1ccccc1-2,CI,0


In [ ]:
hiv_df_sampled = pd.concat([hiv_df_filtered_active, hiv_df_filtered_inactive], axis=0, ignore_index=True)
hiv_df_sampled

,smiles,activity,HIV_active
0,O=C(O)Cc1ccc(SSc2ccc(CC(=O)O)cc2)cc1,CM,1
1,NNP(=S)(NN)c1ccccc1,CM,1
2,O=Nc1ccc(O)c(N=O)c1O,CM,1
3,Oc1ccc(Cl)cc1C(c1cc(Cl)ccc1O)C(Cl)(Cl)Cl,CM,1
4,NNC(=O)c1ccccc1SSc1ccccc1C(=O)NN,CM,1
...,...,...,...
2938,CC(=O)OC1(C#N)CC2OC1C1C2N1C(=O)OC(C)(C)C,CI,0
2939,CCOC(=O)C1Cc2cc(C)c(C)cc2N(C)C1=O,CI,0
2940,CCOC(=O)N1CCN(c2ccc3c(C)cc(C)nc3n2)CC1,CI,0
2941,CN(C)C=Nc1ccc2c3c(cccc13)-c1ccccc1-2,CI,0


In [ ]:
# Randomly shuffle rows
hiv_df_sampled = hiv_df_sampled.sample(frac=1, random_state=42)

In [ ]:
hiv_df_sampled.head()
hiv_df.info()

,smiles,activity,HIV_active
840,C[N+](C)(C)C.O=C(Nc1ccc(C=Cc2ccc(NC(=O)c3cc(S(...,CM,1
1037,COC(=O)Cc1cc(O)c(CC=C(C)CCC=C(C)C(O)C(=O)C=C(C...,CM,1
2399,CCCCCCCCCCC#CCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C...,CI,0
678,Cc1cc(C)c(S(=O)(O)=[OH+])c(C)c1.N[S+]1Cc2nc3cc...,CM,1
196,CCOc1cc(C2=NN3C(=S)NNC3=NN2)ccc1O,CM,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41127 entries, 0 to 41126
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   smiles      41127 non-null  object
 1   activity    41127 non-null  object
 2   HIV_active  41127 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 964.0+ KB


In [ ]:
hiv_df_sampled.to_csv('HIV_2.csv', index=False)
# .drop(['activity'], axis=1).
hiv_df_sampled_2 = pd.read_csv("HIV_2.csv")
hiv_df_sampled_2.head()
hiv_df_sampled_2.tail()

,smiles,activity,HIV_active
0,C[N+](C)(C)C.O=C(Nc1ccc(C=Cc2ccc(NC(=O)c3cc(S(...,CM,1
1,COC(=O)Cc1cc(O)c(CC=C(C)CCC=C(C)C(O)C(=O)C=C(C...,CM,1
2,CCCCCCCCCCC#CCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C...,CI,0
3,Cc1cc(C)c(S(=O)(O)=[OH+])c(C)c1.N[S+]1Cc2nc3cc...,CM,1
4,CCOc1cc(C2=NN3C(=S)NNC3=NN2)ccc1O,CM,1


,smiles,activity,HIV_active
2938,CCCCCCCCNC(=S)NC=C1C(=O)Oc2ccccc2C1=O,CI,0
2939,COP(=O)(C=Cc1cc(C(=O)OCc2ccccc2)n(S(=O)(=O)c2c...,CM,1
2940,O=C(CCc1ccc(O)cc1)NCCc1ccc(O)cc1,CM,1
2941,CCc1c(Cc2cc(C)cc(C)c2)n(COCCCO)c(=O)[nH]c1=O,CA,1
2942,CCSCCCCCCCCCCC(=O)OCC1OC(n2cc(C)c(=O)[nH]c2=O)...,CA,1


In [ ]:
hiv_df_sampled_2

,smiles,activity,HIV_active
0,C[N+](C)(C)C.O=C(Nc1ccc(C=Cc2ccc(NC(=O)c3cc(S(...,CM,1
1,COC(=O)Cc1cc(O)c(CC=C(C)CCC=C(C)C(O)C(=O)C=C(C...,CM,1
2,CCCCCCCCCCC#CCC(=O)SCCNC(=O)CCNC(=O)C(O)C(C)(C...,CI,0
3,Cc1cc(C)c(S(=O)(O)=[OH+])c(C)c1.N[S+]1Cc2nc3cc...,CM,1
4,CCOc1cc(C2=NN3C(=S)NNC3=NN2)ccc1O,CM,1
...,...,...,...
2938,CCCCCCCCNC(=S)NC=C1C(=O)Oc2ccccc2C1=O,CI,0
2939,COP(=O)(C=Cc1cc(C(=O)OCc2ccccc2)n(S(=O)(=O)c2c...,CM,1
2940,O=C(CCc1ccc(O)cc1)NCCc1ccc(O)cc1,CM,1
2941,CCc1c(Cc2cc(C)cc(C)c2)n(COCCCO)c(=O)[nH]c1=O,CA,1


In [ ]:
arguments = [
    '--data_path', 'HIV_2.csv',
    '--dataset_type', 'classification',
    '--save_dir', 'test_checkpoints_multimolecule',
    '--epochs', '30',
    '--save_smiles_splits',
    '--quiet',
    '--batch_size', '64',
    '--ignore_columns', 'activity',
    '--depth', '5',
    # '--hidden_size', '1600',
    '--dropout', '0.2',
]

args = chemprop.args.TrainArgs().parse_args(arguments)

In [ ]:
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

2943it [00:00, 204104.58it/s]
100%|██████████| 2943/2943 [00:00<00:00, 102805.34it/s]
Fold 0
2943it [00:00, 283733.75it/s]
100%|██████████| 37/37 [00:03<00:00,  8.77it/s]
                                               
100%|██████████| 37/37 [00:04<00:00, 10.66it/s]
                                               
 97%|█████████▋| 36/37 [00:04<00:00, 10.56it/s]
                                               
100%|██████████| 37/37 [00:04<00:00,  5.46it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00, 10.21it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00, 10.62it/s]
                                               
 97%|█████████▋| 36/37 [00:05<00:00,  6.78it/s]
                                               
 97%|█████████▋| 36/37 [00:03<00:00,  9.06it/s]
                                               
100%|██████████| 37/37 [00:03<00:00, 10.71it/s]
                                               
 97%|████████

In [ ]:
mean_score, std_score

(0.8445180889257111, 0.0)

In [ ]:
bp_df = pd.read_csv("BBBP.csv")
bp_df.head()
bp_df.info()

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   num     2050 non-null   int64 
 1   name    2050 non-null   object
 2   p_np    2050 non-null   int64 
 3   smiles  2050 non-null   object
dtypes: int64(2), object(2)
memory usage: 64.2+ KB


In [ ]:
bp_df.tail()

,num,name,p_np,smiles
2045,2049,licostinel,1,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl
2046,2050,ademetionine(adenosyl-methionine),1,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...
2047,2051,mesocarb,1,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...
2048,2052,tofisoline,1,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...
2049,2053,azidamfenicol,1,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...


In [ ]:
bp_df.drop(['num', 'name', 'p_np'], axis=1).to_csv('BBBP_2.csv', index=False)

In [ ]:
bp_df_2 = pd.read_csv("BBBP_2.csv")
bp_df_2.tail()
bp_df_2.info()

,smiles
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...
2049,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   smiles  2050 non-null   object
dtypes: object(1)
memory usage: 16.1+ KB


In [ ]:
arguments = [
    '--test_path', 'BBBP_2.csv',
    '--preds_path', 'BBBP_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


2050it [00:00, 222362.76it/s]
100%|██████████| 2050/2050 [00:00<00:00, 130380.35it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validating SMILES
Test size = 2,039


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:08<00:00,  8.56s/it]

Saving predictions to BBBP_preds.csv
Elapsed time = 0:00:09


In [ ]:
bp_preds_df = pd.read_csv("BBBP_preds.csv")
bp_preds_df.head()

,smiles,HIV_active
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.08995568752288818
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,0.13612912595272064
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,0.2891809046268463
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,0.07175479829311371
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,0.3540500998497009


In [ ]:
bp_preds_df.tail()

,smiles,HIV_active
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,0.28399333357810974
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,0.13631369173526764
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,0.4787648916244507
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,0.33214083313941956
2049,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...,0.479694128036499


In [ ]:
bp_preds_df.describe()

,smiles,HIV_active
count,2050,2050
unique,2050,1997
top,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,Invalid SMILES
freq,1,11


In [ ]:
bp_preds_df = bp_preds_df[bp_preds_df['HIV_active'] != "Invalid SMILES"]
bp_preds_df.describe()

,smiles,HIV_active
count,2039,2039
unique,2039,1996
top,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.06751451641321182
freq,1,3


In [ ]:
bp_preds_df['HIV_active'] = bp_preds_df['HIV_active'].astype(float)

In [ ]:
bp_preds_df['HIV_active_2'] = bp_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.9 else 0)
bp_preds_df

,smiles,HIV_active,HIV_active_2
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,0.089956,0
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,0.136129,0
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,0.289181,0
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,0.071755,0
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,0.354050,0
...,...,...,...
2045,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,0.283993,0
2046,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,0.136314,0
2047,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,0.478765,0
2048,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,0.332141,0


In [ ]:
bp_preds_df.describe()

,HIV_active,HIV_active_2
count,2039.000000,2039.000000
mean,0.294235,0.008337
std,0.234286,0.090950
min,0.000125,0.000000
25%,0.101336,0.000000
50%,0.214237,0.000000
75%,0.452856,0.000000
max,0.999444,1.000000


In [ ]:
# Filter rows where 'target_column' is equal to 1
bp_preds_df_filtered = bp_preds_df[bp_preds_df['HIV_active_2'] == 1]
bp_preds_df_filtered


,smiles,HIV_active,HIV_active_2
11,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O,0.901065,1
319,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O,0.955664,1
427,CC(C)[C@@H]1NC(=O)[C@H](C)OC(=O)C(NC(=O)[C@H](...,0.951809,1
445,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,0.991337,1
527,CN(C)CC\C=C/1c2ccccc2Sc3ccc(Cl)cc13,0.951663,1
912,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(NC(=...,0.939820,1
945,Oc1ccc(cc1)/C=C([N+]#[C-])/C(=C/c2ccc(O)cc2)[N...,0.956967,1
973,COC1(C(F)(F)C1(F)Cl)F,0.986301,1
1004,C(C1(C(NC(=O)NC1=O)=O)C(C)C)C=C,0.940815,1
1165,CC1(C(NC(O1)=O)=O)C,0.903444,1


In [ ]:
smiles_to_check = bp_preds_df_filtered['smiles'].to_list()

In [ ]:
hiv_df_sampled_2[hiv_df_sampled_2['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [ ]:
hiv_df[hiv_df['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [ ]:
bp_df[bp_df['smiles'].isin(smiles_to_check)]

,num,name,p_np,smiles
11,12,alovudine,1,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O
319,321,Stavudine,1,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O
427,429,valinomycin,0,CC(C)[C@@H]1NC(=O)[C@H](C)OC(=O)C(NC(=O)[C@H](...
445,447,zidovudine,0,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...
527,529,chlorprothixene,1,CN(C)CC\C=C/1c2ccccc2Sc3ccc(Cl)cc13
912,914,rifamycin,0,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(NC(=...
945,947,xantocillin,0,Oc1ccc(cc1)/C=C([N+]#[C-])/C(=C/c2ccc(O)cc2)[N...
973,975,aliflurane,1,COC1(C(F)(F)C1(F)Cl)F
1004,1006,aprobarbital,1,C(C1(C(NC(=O)NC1=O)=O)C(C)C)C=C
1165,1169,dimethadione,1,CC1(C(NC(O1)=O)=O)C


In [ ]:
bp_df_final = pd.merge(bp_df[bp_df['smiles'].isin(smiles_to_check)], bp_preds_df_filtered, on='smiles' )
bp_df_final

,num,name,p_np,smiles,HIV_active,HIV_active_2
0,12,alovudine,1,CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O,0.901065,1
1,321,Stavudine,1,CC1=CN([C@@H]2O[C@H](CO)C=C2)C(=O)NC1=O,0.955664,1
2,429,valinomycin,0,CC(C)[C@@H]1NC(=O)[C@H](C)OC(=O)C(NC(=O)[C@H](...,0.951809,1
3,447,zidovudine,0,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,0.991337,1
4,529,chlorprothixene,1,CN(C)CC\C=C/1c2ccccc2Sc3ccc(Cl)cc13,0.951663,1
5,914,rifamycin,0,CO[C@H]1/C=C/O[C@@]2(C)Oc3c(C)c(O)c4c(O)c(NC(=...,0.939820,1
6,947,xantocillin,0,Oc1ccc(cc1)/C=C([N+]#[C-])/C(=C/c2ccc(O)cc2)[N...,0.956967,1
7,975,aliflurane,1,COC1(C(F)(F)C1(F)Cl)F,0.986301,1
8,1006,aprobarbital,1,C(C1(C(NC(=O)NC1=O)=O)C(C)C)C=C,0.940815,1
9,1169,dimethadione,1,CC1(C(NC(O1)=O)=O)C,0.903444,1


In [ ]:
bp_df_final.to_csv('HIV_result.csv', index=False)

In [ ]:
sub_df = pd.read_csv("substances.csv")
sub_df.head()

,zinc_id,smiles
0,ZINC000000000027,N[C@@H](CCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
1,ZINC000016090786,N[C@H](CCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
2,ZINC000001763088,N[C@H](CCCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
3,ZINC000002033385,N[C@@H](CCCc1ccc(N(CCCl)CCCl)cc1)C(=O)O
4,ZINC000000001673,N[C@@H](Cc1ccc(N(CCCl)CCCl)cc1)C(=O)O


In [ ]:
sub_df.tail()

,zinc_id,smiles
46,ZINC000196349655,O=C(O)CCSc1ccc(N(CCCl)CCCl)cc1
47,ZINC000064454242,N=NCCCc1ccc(N(CCCl)CCCl)cc1
48,ZINC000005161807,O=C(O)C/C=C/c1ccc(N(CCCl)CCCl)cc1
49,ZINC000001682294,O=C(O)CCOc1ccc(N(CCCl)CCCl)cc1
50,ZINC000079564304,O=C(O)CNC(=O)c1ccc(N(CCCl)CCCl)cc1


In [ ]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   zinc_id  51 non-null     object
 1   smiles   51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [ ]:
arguments = [
    '--test_path', 'substances.csv',
    '--preds_path', 'substances_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


51it [00:00, 66659.24it/s]
100%|██████████| 51/51 [00:00<00:00, 80026.00it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validating SMILES
Test size = 51


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


Saving predictions to substances_preds.csv
Elapsed time = 0:00:02


In [ ]:
fda_df = pd.read_csv("fda_approved.csv")
fda_df.head()

,zinc_id,smiles
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...
4,ZINC000000008492,Oc1cccc2cccnc12


In [ ]:
arguments = [
    '--test_path', 'fda_approved.csv',
    '--preds_path', 'fda_approved_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


892it [00:00, 191430.58it/s]
100%|██████████| 892/892 [00:00<00:00, 134054.22it/s]

Validating SMILES



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test size = 892


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [00:04<00:00,  4.25s/it]

Saving predictions to fda_approved_preds.csv
Elapsed time = 0:00:05


In [ ]:
fda_preds_df = pd.read_csv("fda_approved_preds.csv")
fda_preds_df.head()
fda_preds_df.info()

,zinc_id,smiles,HIV_active
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O,0.062741
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.871797
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1,0.128321
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0.582539
4,ZINC000000008492,Oc1cccc2cccnc12,0.373629


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   zinc_id     892 non-null    object 
 1   smiles      892 non-null    object 
 2   HIV_active  892 non-null    float64
dtypes: float64(1), object(2)
memory usage: 21.0+ KB


In [ ]:
fda_preds_df = fda_preds_df[fda_preds_df['HIV_active'] != "Invalid SMILES"]
fda_preds_df.describe()
fda_preds_df['HIV_active'] = fda_preds_df['HIV_active'].astype(float)
fda_preds_df['HIV_active_2'] = fda_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.9 else 0)
fda_preds_df.head()

,HIV_active
count,892.000000
mean,0.357054
std,0.223882
min,0.003678
25%,0.188324
50%,0.303072
75%,0.496570
max,0.998864


,zinc_id,smiles,HIV_active,HIV_active_2
0,ZINC000001530427,C[C@@H]1O[C@@H]1P(=O)(O)O,0.062741,0
1,ZINC000003807804,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,0.871797,0
2,ZINC000000120286,Nc1nc(N)c2nc(-c3ccccc3)c(N)nc2n1,0.128321,0
3,ZINC000242548690,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,0.582539,0
4,ZINC000000008492,Oc1cccc2cccnc12,0.373629,0


In [ ]:
# Filter rows where 'target_column' is equal to 1
fda_preds_df_filtered = fda_preds_df[fda_preds_df['HIV_active_2'] == 1]
fda_preds_df_filtered

,zinc_id,smiles,HIV_active,HIV_active_2
8,ZINC000051133897,CN1C(C(=O)Nc2ccccn2)=C(O)c2ccccc2S1(=O)=O,0.942730,1
31,ZINC000003816287,CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1,0.902933,1
81,ZINC000003818726,O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO,0.949465,1
94,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,0.990190,1
131,ZINC000000020231,Nc1cc(Cl)c(N=C2NCCN2)c(Cl)c1,0.944310,1
134,ZINC000000896484,Clc1cccc(Cl)c1N=C1NCCN1,0.966284,1
228,ZINC000009302239,NC(=O)[C@@H]1CC[C@@H]2CN1C(=O)N2OS(=O)(=O)O,0.958752,1
267,ZINC000003830999,N[C@@H](Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1)C(=O)O,0.912747,1
276,ZINC000000002212,Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2,0.939824,1
352,ZINC000000020253,NS(=O)(=O)c1cc([C@]2(O)NC(=O)c3ccccc32)ccc1Cl,0.941636,1


In [ ]:
smiles_to_check = fda_preds_df_filtered['smiles'].to_list()
print(f"smiles to check: {smiles_to_check}")

smiles to check: ['CN1C(C(=O)Nc2ccccn2)=C(O)c2ccccc2S1(=O)=O', 'CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1', 'O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO', 'CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1', 'Nc1cc(Cl)c(N=C2NCCN2)c(Cl)c1', 'Clc1cccc(Cl)c1N=C1NCCN1', 'NC(=O)[C@@H]1CC[C@@H]2CN1C(=O)N2OS(=O)(=O)O', 'N[C@@H](Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1)C(=O)O', 'Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2', 'NS(=O)(=O)c1cc([C@]2(O)NC(=O)c3ccccc32)ccc1Cl', 'COc1cc(/C(O)=N/S(=O)(=O)c2ccccc2C)ccc1Cc1cn(C)c2ccc(NC(=O)OC3CCCC3)cc12', 'CCCS(=O)(=O)Nc1ccc(F)c(C(=O)c2c[nH]c3ncc(-c4ccc(Cl)cc4)cc23)c1F', 'C[C@H](CN1c2ccccc2Sc2ccccc21)N(C)C', 'N=C1CCCN1Cc1[nH]c(=O)[nH]c(=O)c1Cl', 'C[C@]12CC[C@@H]3c4ccc(OS(=O)(=O)O)cc4CC[C@H]3[C@@H]1CCC2=O', 'N[C@@H](Cc1cc(I)c(Oc2cc(I)c(O)c(I)c2)c(I)c1)C(=O)O', 'O=c1[nH]cc(F)c(=O)[nH]1', 'N=C(N)Nc1nc(CSCCC(=N)NS(N)(=O)=O)cs1', 'C[C@@H](CN1c2ccccc2Sc2ccccc21)N(C)C', 'Cc1cnc(NC(=O)C2=C(O)c3ccccc3S(=O)(=O)N2C)s1', 'Cc1cc(-c2ccc(/N=N/c3c(S(=O)(=O)O)cc4

In [ ]:
hiv_df_sampled_2[hiv_df_sampled_2['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [ ]:
hiv_df[hiv_df['smiles'].isin(smiles_to_check)]

,smiles,activity,HIV_active


In [ ]:
bp_df[bp_df['smiles'].isin(smiles_to_check)]

,num,name,p_np,smiles


In [ ]:
fda_df[fda_df['smiles'].isin(smiles_to_check)]

,zinc_id,smiles
8,ZINC000051133897,CN1C(C(=O)Nc2ccccn2)=C(O)c2ccccc2S1(=O)=O
31,ZINC000003816287,CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1
81,ZINC000003818726,O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO
94,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...
131,ZINC000000020231,Nc1cc(Cl)c(N=C2NCCN2)c(Cl)c1
134,ZINC000000896484,Clc1cccc(Cl)c1N=C1NCCN1
228,ZINC000009302239,NC(=O)[C@@H]1CC[C@@H]2CN1C(=O)N2OS(=O)(=O)O
267,ZINC000003830999,N[C@@H](Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1)C(=O)O
276,ZINC000000002212,Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2
352,ZINC000000020253,NS(=O)(=O)c1cc([C@]2(O)NC(=O)c3ccccc32)ccc1Cl


In [ ]:
fda_df_final = pd.merge(fda_df[fda_df['smiles'].isin(smiles_to_check)], fda_preds_df_filtered, on='smiles' )
fda_df_final

,zinc_id_x,smiles,zinc_id_y,HIV_active,HIV_active_2
0,ZINC000051133897,CN1C(C(=O)Nc2ccccn2)=C(O)c2ccccc2S1(=O)=O,ZINC000051133897,0.942730,1
1,ZINC000003816287,CNC(=O)c1ccccc1Sc1ccc2c(/C=C/c3ccccn3)n[nH]c2c1,ZINC000003816287,0.902933,1
2,ZINC000003818726,O=C(/C=C/c1cccc(S(=O)(=O)Nc2ccccc2)c1)NO,ZINC000003818726,0.949465,1
3,ZINC000068153186,CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2...,ZINC000068153186,0.990190,1
4,ZINC000000020231,Nc1cc(Cl)c(N=C2NCCN2)c(Cl)c1,ZINC000000020231,0.944310,1
5,ZINC000000896484,Clc1cccc(Cl)c1N=C1NCCN1,ZINC000000896484,0.966284,1
6,ZINC000009302239,NC(=O)[C@@H]1CC[C@@H]2CN1C(=O)N2OS(=O)(=O)O,ZINC000009302239,0.958752,1
7,ZINC000003830999,N[C@@H](Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1)C(=O)O,ZINC000003830999,0.912747,1
8,ZINC000000002212,Cc1nnc2n1-c1ccc(Cl)cc1C(c1ccccc1Cl)=NC2,ZINC000000002212,0.939824,1
9,ZINC000000020253,NS(=O)(=O)c1cc([C@]2(O)NC(=O)c3ccccc32)ccc1Cl,ZINC000000020253,0.941636,1


In [ ]:
fda_df_final.to_csv('fda_approved_result.csv', index=False)

In [ ]:
# !wget https://zinc15.docking.org/substances/subsets/named.csv

--2024-03-10 05:58:28--  https://zinc15.docking.org/substances/subsets/named.csv
Resolving zinc15.docking.org (zinc15.docking.org)... 169.230.75.4
Connecting to zinc15.docking.org (zinc15.docking.org)|169.230.75.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘named.csv.1’

named.csv.1             [ <=>                ]   9.28K  --.-KB/s    in 0.04s   

2024-03-10 05:58:29 (242 KB/s) - ‘named.csv.1’ saved [9499]



In [ ]:
zinc_df = pd.read_csv("named.csv")
zinc_df.head()
zinc_df.tail()
zinc_df.info()

,zinc_id,smiles
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...


,zinc_id,smiles
34595,ZINC000005999135,COc1cc([C@@H]2Oc3c(OC)cc(/C=C/CO)cc3[C@H]2CO)c...
34596,ZINC000084710404,COC(=O)c1cc(OC)c2cc(OC)c(OC)c(O)c2c1O
34597,ZINC000150369761,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)O[...
34598,ZINC000095098911,CCCCCCCCCCCC[C@H](O)[C@H]1CC[C@H]([C@H](O)CCCC...
34599,ZINC000000001009,C[N+](C)([O-])CC/C=C1\c2ccccc2C=Cc2c(Cl)cccc21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34600 entries, 0 to 34599
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   zinc_id  34600 non-null  object
 1   smiles   34600 non-null  object
dtypes: object(2)
memory usage: 540.8+ KB


In [ ]:
arguments = [
    '--test_path', 'named.csv',
    '--preds_path', 'named_preds.csv',
    '--checkpoint_dir', 'test_checkpoints_multimolecule',
    '--smiles_columns', 'smiles'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


34600it [00:00, 52369.15it/s]
100%|██████████| 34600/34600 [00:00<00:00, 46847.06it/s]


Validating SMILES


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test size = 34,600


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda



100%|██████████| 1/1 [02:39<00:00, 159.07s/it]


Saving predictions to named_preds.csv
Elapsed time = 0:02:53


In [ ]:
zinc_preds_df = pd.read_csv("named_preds.csv")
zinc_preds_df.head()
zinc_preds_df = zinc_preds_df[zinc_preds_df['HIV_active'] != "Invalid SMILES"]
zinc_preds_df.describe()
zinc_preds_df['HIV_active'] = zinc_preds_df['HIV_active'].astype(float)
zinc_preds_df['HIV_active_2'] = zinc_preds_df['HIV_active'].apply(lambda x: 1 if x > 0.9 else 0)
zinc_preds_df.head()

,zinc_id,smiles,HIV_active
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,0.397770
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,0.594493
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,0.305740
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,0.218365
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,0.668774


,HIV_active
count,34600.000000
mean,0.503277
std,0.225115
min,0.000016
25%,0.320969
50%,0.512118
75%,0.702356
max,0.999995


,zinc_id,smiles,HIV_active,HIV_active_2
0,ZINC000030727788,C=C[C@]1(C)C[C@@H](OC(=O)CSC(C)(C)CNC(=O)[C@H]...,0.397770,0
1,ZINC000150377216,CCCCCC/C=C\C/C=C\CCCCCCCC(=O)OC[C@H](COCCCCCCC...,0.594493,0
2,ZINC000100780125,CC(=O)O[C@H]1C[C@](C)(O)[C@@H]2CC=C(C)[C@@H]2[...,0.305740,0
3,ZINC000006580536,O=C(O)[C@H](Cc1ccccc1)N(CCCl)CCCl,0.218365,0
4,ZINC000150351802,O=C1C[C@H](c2ccc(O)c(O)c2)Oc2c1c(O)cc(O[C@H]1O...,0.668774,0


In [ ]:
# Filter rows where 'target_column' is equal to 1
zinc_preds_df_filtered = zinc_preds_df[zinc_preds_df['HIV_active_2'] == 1]
zinc_preds_df_filtered

,zinc_id,smiles,HIV_active,HIV_active_2
128,ZINC000008220075,Nc1nc(O)c(N)c(N[C@@H]2O[C@H](CO[P@@](=O)(O)O[P...,0.972243,1
149,ZINC000014680924,Cc1c(Cl)c(O)cc(O)c1C(=O)O[C@H]1C[C@]2(C)[C@@H]...,0.921616,1
194,ZINC000082170194,O=C(OC[C@H]1OC[C@H](OC(=O)c2cc(O)c(O)c(O)c2)[C...,0.958125,1
218,ZINC000033979236,O=c1ccn([C@H]2C[C@@H](O)[C@H](CO[P@@](=O)(O)OP...,0.976609,1
232,ZINC000049089629,C[C@@H]1CC(=O)[C@H]2C[C@]1(O)[C@]2(C)COC(=O)c1...,0.953318,1
...,...,...,...,...
34381,ZINC000103285060,CCCCCCOC(=O)c1cc(O)cc(O)c1,0.930157,1
34386,ZINC000150368025,O=C(/C=C/c1ccc(O)cc1)O[C@@H]1[C@H](OC(=O)c2cc(...,0.930840,1
34408,ZINC000150340622,N[C@@H](Cc1cc([125I])c(Oc2ccc(O)c([125I])c2)c(...,0.911913,1
34537,ZINC000005759355,CC(C)COC(=O)c1ccccc1C(=O)O,0.962571,1


In [ ]:
zinc_preds_df_filtered.to_csv('zinc_final_result.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir '/content/drive/My Drive/Chemprop_Backup_4/'

In [ ]:
!pwd

/content


In [ ]:
!ls -al

total 14336
drwxr-xr-x 1 root root    4096 Mar 11 10:59 .
drwxr-xr-x 1 root root    4096 Mar 11 10:10 ..
-rw-r--r-- 1 root root  107579 Mar 11 10:26 BBBP_2.csv
-rw-r--r-- 1 root root  148743 Mar 11 10:18 BBBP.csv
-rw-r--r-- 1 root root  149427 Mar 11 10:26 BBBP_preds.csv
drwxr-xr-x 4 root root    4096 Mar  7 14:31 .config
drwx------ 6 root root    4096 Mar 11 10:59 drive
-rw-r--r-- 1 root root   61865 Mar 11 10:18 fda_approved.csv
-rw-r--r-- 1 root root   79192 Mar 11 10:45 fda_approved_preds.csv
-rw-r--r-- 1 root root    2514 Mar 11 10:52 fda_approved_result.csv
-rw-r--r-- 1 root root  174576 Mar 11 10:35 HIV_2.csv
-rw-r--r-- 1 root root 2193844 Mar 11 10:18 HIV.csv
-rw-r--r-- 1 root root    2181 Mar 11 10:45 HIV_result.csv
drwxr-xr-x 4 root root    4096 Mar 11 10:15 Molecule-Classification
-rw-r--r-- 1 root root 4622724 Mar 11 10:18 musk.csv
-rw-r--r-- 1 root root 3160219 Mar 11 10:18 named.csv
-rw-r--r-- 1 root root 3824043 Mar 11 10:55 named_preds.csv
drwxr-xr-x 1 root root    4096

In [ ]:
!cp -r /content/*.csv "/content/drive/My Drive/Chemprop_Backup_4/"

In [ ]:
!cp -r /content/test_checkpoints_multimolecule/ "/content/drive/My Drive/Chemprop_Backup_4/"